In [161]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import display, HTML
from arc import *

Some examples of functions provided by the ARC package.

In [163]:
atom = Rubidium85()
n1 = 5
l1 = 0
j1 = 0.5

n2 = 6
l2 = 1
j2 = 1.5

q = +1

print("Radial matrix element:", atom.getRadialMatrixElement(n1, l1, j1, n2, l2, j2))
print("Radial matrix element:", atom.getRadialMatrixElement(n2, l2, j2, n1, l1, j1))

print("Reduced matrix element L:", atom.getReducedMatrixElementL(n1, l1, j1, n2, l2, j2))
print("Reduced matrix element L:", atom.getReducedMatrixElementL(n2, l2, j2, n1, l1, j1))

print("Reduced matrix element J:", atom.getReducedMatrixElementJ(n1, l1, j1, n2, l2, j2))
print("Reduced matrix element J:", atom.getReducedMatrixElementJ(n2, l2, j2, n1, l1, j1))

print("Dipole matrix element:", atom.getDipoleMatrixElement(n1, l1, j1, j1, n2, l2, j2, j2, +q))
print("Dipole matrix element:", atom.getDipoleMatrixElement(n2, l2, j2, j2, n1, l1, j1, j1, -q))

print(printStateStringLatex(n1, l1, j1), "<->", printStateStringLatex(n2, l2, j2))

print("λ:", atom.getTransitionWavelength(n1, l1, j1, n2, l2, j2)/1e-9, "nm")
print("λ:", atom.getTransitionWavelength(n2, l2, j2, n1, l1, j1)/1e-9, "nm")

print("f:", atom.getTransitionFrequency(n1, l1, j1, n2, l2, j2)/1e12, "THz")
print("f:", atom.getTransitionFrequency(n2, l2, j2, n1, l1, j1)/1e12, "THz")

print("τ1:", atom.getStateLifetime(n1, l1, j1)/1e-9, "ns")

print("E:", atom.getEnergy(16, 1, 0.5))


print("Spontaneous radiation rate:", atom.getTransitionRate(30, 0, 0.5, 3, 1, 0.5, temperature=300))

Radial matrix element: -0.46851974344738134
Radial matrix element: -0.46851974344738134
Reduced matrix element L: 0.4685197434473813
Reduced matrix element L: -0.4685197434473813
Reduced matrix element J: 0.541
Reduced matrix element J: -0.541
Dipole matrix element: 0.2705
Dipole matrix element: -0.2705
5S_{1/2} <-> 6P_{3/2}
λ: 420.2989249729119 nm
λ: -420.2989249729119 nm
f: 713.2839038770359 THz
f: -713.2839038770359 THz
τ1: 1e+59 ns
E: -0.07641510412387363
Spontaneous radiation rate: 9174.0517887993


Calculation of absorption spectrum. The results are stored in file "absorption.dat"

In [183]:
def check_selection_rule(l1, l2, j1, j2):
    return check_selection_rule_wigner_6j(j1, 1, j2, l2, 0.5, l1)

def check_selection_rule_wigner_6j(j1, j2, j3, J1, J2, J3):
    if ((abs(j1 - j2) > j3)
        | (j1 + j2 < j3)
        | (abs(j1 - J2) > J3)
        | (j1 + J2 < J3)
        | (abs(J1 - j2) > J3)
        | (J1 + j2 < J3)
        | (abs(J1 - J2) > j3)
        | (J1 + J2 < j3)
            ):
        return -1

    # Check if the sum of the elements of each traid is an integer
    if ((2 * (j1 + j2 + j3) != round(2 * (j1 + j2 + j3)))
        | (2 * (j1 + J2 + J3) != round(2 * (j1 + J2 + J3)))
        | (2 * (J1 + j2 + J3) != round(2 * (J1 + j2 + J3)))
        | (2 * (J1 + J2 + j3) != round(2 * (J1 + J2 + j3)))
            ):
        return -1
    return 1

def get_dipole_moment(n1, l1, j1, mj1, n2, l2, j2, mj2, q):
    if q != 0:
        if mj2 - mj1 != q:
            return 0
        if check_selection_rule(l1, l2, j1, j2) < 0:
            return 0
        return atom.getDipoleMatrixElement(n1, l1, j1, mj1, n2, l2, j2, mj2, q)
    else:
        if j1 == 0 and j2 == 0:
            return 0
        if mj2 - mj1 != q:
            return 0
        if check_selection_rule(l1, l2, j1, j2) < 0:
            return 0
        return atom.getDipoleMatrixElement(n1, l1, j1, mj1, n2, l2, j2, mj2, q)


n_max = 30 # 40
atom = Rubidium85()
q = +1
    
with open("absorption.dat", "w") as f:
    for n1 in range(5, n_max):
        for l1 in range(0, 4):
            for n2 in range(n1, n_max):
                for l2 in range(0, 4):
                    
                    if n1 == n2 and l1 == l2:
                        continue

                    j1_list = []
                    j2_list = []

                    if l1 > 0:
                        j1_list = [l1-0.5, l1+0.5]
                    else:
                        j1_list = [0.5]

                    if l2 > 0:
                        j2_list = [l2-0.5, l2+0.5]
                    else:
                        j2_list = [0.5]

                    for j1 in j1_list:
                        for j2 in j2_list:
                            frequency = atom.getTransitionFrequency(n1, l1, j1, n2, l2, j2) / 1e12
                            #dipole = atom.getReducedMatrixElementL(n1, l1, j1, n2, l2, j2)

                            mj1_list = np.arange(-j1, j1+1, 1)
                            mj2_list = np.arange(-j2, j2+1, 1)
                            
                            for mj1 in mj1_list:
                                for mj2 in mj2_list:
                                    if frequency > 0:
                                        dipole = get_dipole_moment(n1, l1, j1, mj1, n2, l2, j2, mj2, q)
                                    else:
                                        dipole = get_dipole_moment(n2, l2, j2, mj2, n1, l1, j1, mj1, -q)
                                
                                    if np.abs(dipole) > 1e-5:
                                            wavelength = atom.getTransitionWavelength(n1, l1, j1, n2, l2, j2)/1e-9
                                            #f.write("%3d %3d %5.1f %5.1f %3d %3d %5.1f %5.1f  %le  %le  %le  %s <-> %s\n" % \
                                            #        (n1, l1, j1, mj1, n2, l2, j2, mj2, \
                                            #         frequency, wavelength, dipole,  \
                                            #         printStateStringLatex(n1, l1, j1), \
                                            #         printStateStringLatex(n2, l2, j2)))
                                            f.write("%3d %3d %5.1f %5.1f %3d %3d %5.1f %5.1f  %le  %le  %le\n" % \
                                                    (n1, l1, j1, mj1, n2, l2, j2, mj2, \
                                                     frequency, wavelength, dipole))

Export information of energy levels, including energy and lifetime, to file "levels.dat"

In [184]:
n_max = 30
l_max = 4
atom = Rubidium85()

with open("levels.dat", "w") as f:
    for n in range(5, n_max):
        for l in range(0, l_max):
            j_list = []
            if l > 0:
                j_list = [l-0.5, l+0.5]
            else:
                j_list = [0.5]
            for j in j_list:
                energy = atom.getEnergy(n, l, j)
                lifetime = atom.getStateLifetime(n, l, j)/1e-9
                f.write("%3d %3d %5.1f  %le  %le\n" % (n, l, j, energy, lifetime))

Test with the calcuation of spontaneous radiation

In [168]:
atom = Rubidium85()

n1 = 30
l1 = 0
j1 = 0.5

n2 = 3
l2 = 1
j2 = 0.5

temperature = 300


rate = atom.getTransitionRate(n1, l1, j1, n2, l2, j2, temperature=300)
print(1. / rate / 1e-6, "us")

rate = atom.getTransitionRate(n2, l2, j2, n1, l1, j1, temperature=300)
print(1. / rate / 1e-6, "us")

109.0030907849148 us
inf us


/home/tianmin/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


Calculation of spontaneous radiation, recording all channels in file "spontaneous.dat"

In [185]:
n_max = 30
l_max = 4
atom = Rubidium85()
temperature = 300

def check_spontaneous_transition_rule(l1, l2, j1, j2):
    return check_selection_rule_wigner_6j(l1, l2, 1, j2, j1, 0.5)

with open("spontaneous.dat", "w") as f:
    for n1 in range(5, n_max):
        for n2 in range(5, n_max):
            for l1 in range(0, l_max):
                for l2 in range(0, l_max):

                    j1_list = []
                    j2_list = []

                    if l1 > 0:
                        j1_list = [l1-0.5, l1+0.5]
                    else:
                        j1_list = [0.5]

                    if l2 > 0:
                        j2_list = [l2-0.5, l2+0.5]
                    else:
                        j2_list = [0.5]

                    for j1 in j1_list:
                        for j2 in j2_list:
                            if n1==n2 and l1==l2 and j1==j2:
                                continue
                            frequency = atom.getTransitionFrequency(n1, l1, j1, n2, l2, j2)/1e12
                            wavelength = atom.getTransitionWavelength(n1, l1, j1, n2, l2, j2)/1e-9
                            if frequency < 0: # E1 > E2
                                if check_spontaneous_transition_rule(l1, l2, j1, j2) > 0:
                                    rate = atom.getTransitionRate(n1, l1, j1, n2, l2, j2, temperature=temperature)
                                    if rate > 1e-5:
                                        f.write("%3d %3d %5.1f %3d %3d %5.1f  %le  %le  %le\n" % \
                                            (n1, l1, j1, n2, l2, j2, -frequency, -wavelength, rate))                            
                  
                